In [2]:
import pandas as pd
import numpy as np

Loading in the dataset

In [3]:
data_train = pd.read_csv("./fer2013.csv")
data_train.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
# make the datset to an array of images of pixels 
image_array =[]
for i, row in enumerate(data_train.index):
        image = np.fromstring(data_train.loc[row, 'pixels'], dtype=int, sep=' ')
        image_array.append(image.flatten())
image_array

[array([ 70,  80,  82, ..., 106, 109,  82]),
 array([151, 150, 147, ..., 193, 183, 184]),
 array([231, 212, 156, ...,  88, 110, 152]),
 array([ 24,  32,  36, ..., 142, 143, 142]),
 array([ 4,  0,  0, ..., 30, 29, 30]),
 array([55, 55, 55, ..., 34, 30, 57]),
 array([ 20,  17,  19, ...,  99, 107, 118]),
 array([ 77,  78,  79, ..., 125,  67,  68]),
 array([85, 84, 90, ..., 58, 73, 84]),
 array([255, 254, 255, ..., 254, 255, 255]),
 array([ 30,  24,  21, ..., 172, 173, 173]),
 array([39, 75, 78, ..., 84, 83, 87]),
 array([219, 213, 206, ...,   0,   0,   0]),
 array([148, 144, 130, ..., 112, 111, 111]),
 array([ 4,  2, 13, ...,  3,  7, 12]),
 array([107, 107, 109, ...,  83,  84, 106]),
 array([14, 14, 18, ...,  9, 10, 10]),
 array([255, 255, 255, ...,  79,  79,  83]),
 array([134, 124, 167, ...,  34,  28, 139]),
 array([219, 192, 179, ...,  57,  39,  40]),
 array([  1,   1,   1, ..., 213, 211, 215]),
 array([174,  51,  37, ...,   7,  11,  16]),
 array([123, 125, 124, ...,  97,  91,  89]),
 

In [6]:
print(f"{len(image_array)} images in the dataset")
print(f"Each image has {len(image_array[0])} pixels")

35887 images in the dataset
Each image has 2304 pixels


In [7]:
lables = np.array(data_train['emotion']).tolist()
lables

[0,
 0,
 2,
 4,
 6,
 2,
 4,
 3,
 3,
 2,
 0,
 6,
 6,
 6,
 3,
 5,
 3,
 2,
 6,
 4,
 4,
 2,
 0,
 0,
 3,
 3,
 5,
 0,
 3,
 5,
 3,
 6,
 3,
 2,
 3,
 6,
 6,
 6,
 0,
 5,
 0,
 3,
 4,
 0,
 6,
 2,
 3,
 6,
 2,
 2,
 2,
 5,
 5,
 6,
 4,
 5,
 0,
 3,
 4,
 2,
 6,
 5,
 3,
 4,
 3,
 0,
 4,
 3,
 0,
 5,
 4,
 2,
 4,
 2,
 0,
 6,
 0,
 0,
 5,
 4,
 3,
 5,
 3,
 6,
 2,
 2,
 4,
 4,
 5,
 4,
 0,
 0,
 6,
 0,
 6,
 2,
 2,
 3,
 4,
 3,
 3,
 2,
 6,
 3,
 4,
 3,
 2,
 0,
 2,
 3,
 2,
 3,
 6,
 6,
 3,
 4,
 4,
 0,
 3,
 3,
 0,
 2,
 0,
 0,
 3,
 3,
 0,
 4,
 5,
 2,
 4,
 2,
 5,
 3,
 0,
 6,
 6,
 2,
 2,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 4,
 2,
 2,
 0,
 0,
 2,
 6,
 3,
 4,
 5,
 3,
 0,
 2,
 4,
 5,
 2,
 3,
 4,
 6,
 3,
 4,
 3,
 6,
 0,
 4,
 4,
 4,
 2,
 6,
 5,
 5,
 6,
 5,
 2,
 3,
 2,
 3,
 4,
 0,
 2,
 0,
 3,
 0,
 2,
 2,
 5,
 3,
 5,
 5,
 4,
 3,
 0,
 6,
 5,
 2,
 6,
 0,
 0,
 4,
 5,
 6,
 3,
 5,
 5,
 3,
 2,
 2,
 3,
 3,
 3,
 3,
 6,
 3,
 6,
 3,
 0,
 6,
 3,
 2,
 0,
 6,
 5,
 6,
 0,
 5,
 4,
 3,
 0,
 3,
 0,
 4,
 5,
 4,
 3,
 3,
 0,
 0,
 6,
 2,
 0,
 6,
 3,
 0,
 0,


In [ ]:
flat_images = np.array(image_array)
target = np.array(lables)

# normalization
flat_images = flat_images / 255